<a href="https://colab.research.google.com/github/DJCordhose/ai/blob/master/notebooks/tensorflow/tf_low_level_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Low Level TensorFlow

* https://www.tensorflow.org/guide/low_level_intro

In [1]:
# import and check version
import tensorflow as tf
# normally we would switch off very verbose warnings, but in this case output from tf might be intersting to understand what is going on
# tf.logging.set_verbosity(tf.logging.ERROR)
tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.12.0


In [2]:
# a small sanity check, does tf seem to work ok? 
sess = tf.Session()
hello = tf.constant('Hello TF!')
print(sess.run(hello))
sess.close()

b'Hello TF!'


## First define a computational graph composed of operations and tensors

In [3]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


## Then use a session to execute the graph

In [4]:
# sessions need to be closed in order not to leak ressources, this makes sure close is called in any case
with tf.Session() as sess:
  print(sess.run(total))

7.0


## Graphs can be executed on CPU, GPU, and even TPU

In [5]:
# let's see what compute devices we have available, hopefully a GPU 
# if you do not see it, switch on under Runtime->Change runtime type
with tf.Session() as sess:
  devices = sess.list_devices()
  for d in devices:
      print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:XLA_CPU:0
/job:localhost/replica:0/task:0/device:XLA_GPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [6]:
with tf.device("/device:XLA_CPU:0"):
  with tf.Session() as sess:
    print(sess.run(total))

7.0


## Feeding data to a graph

In [0]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [8]:
with tf.Session() as sess:
  try:
    print(sess.run(z))
  except tf.errors.InvalidArgumentError as iae:
     print(iae.message)

You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[node Placeholder (defined at <ipython-input-7-3b59dde2b9f0>:1)  = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[{{node add_1/_1}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7_add_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]


In [9]:
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: 3.0, y: 4.5}))

7.5


## Reading in data sets

In [10]:
r = tf.random_normal([10, 2])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

with tf.Session() as sess:
  sess.run(iterator.initializer)
  while True:
    try:
      data = sess.run(next_row)
      print(data)
      print(sess.run(z, feed_dict={x: data[0], y: data[1]}))
    except tf.errors.OutOfRangeError:
      break

[0.5567033  0.59676325]
1.1534666
[1.5197729  0.07503483]
1.5948077
[1.1668001 1.7912807]
2.9580808
[0.03675369 2.1832776 ]
2.2200313
[1.202217   0.36507607]
1.567293
[ 0.08671668 -2.465631  ]
-2.3789144
[ 0.21527272 -0.00577643]
0.20949629
[ 1.5193942  -0.69126767]
0.8281265
[0.11737215 0.20017597]
0.31754813
[ 0.7450656  -0.49466598]
0.25039965


## Layers

In [11]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

with tf.Session() as sess:
  try:
    print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))
  except tf.errors.FailedPreconditionError as fpe:
    print(fpe.message)

Attempting to use uninitialized value dense/kernel
	 [[node dense/kernel/read (defined at <ipython-input-11-34fd23289168>:2)  = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense/kernel)]]
	 [[{{node dense/BiasAdd/_3}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_12_dense/BiasAdd", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]


In [12]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-0.82065153]
 [ 0.34443474]]


In [17]:
y = tf.layers.dense(x, units=2, activation=tf.nn.tanh)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-0.18206427 -0.61147606]
 [-0.92939186  0.23843162]]
